# Lien overleaf rapport :
https://fr.overleaf.com/7578725276tqpwyvhqtmtm

https://towardsdatascience.com/10-tips-for-a-better-google-colab-experience-33f8fe721b82

In [1]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-6a7f5d93-1788-024a-9fce-b6c66efa0376)


#github utiles :


https://github.com/dacostaHugo/computerGo

https://github.com/ReHoss/ResNet_Go

https://github.com/hippolytelrm/ComputerGo

https://github.com/Raphaaal/computer-go

https://adspassets.blob.core.windows.net/website/content/alpha_go_zero_cheat_sheet.png

# NOTES du 11/12/2021

* Alors j'ai implémenté mobilenetV2 du papier de Cazou + ajout du cosine annealing dans le callback pour faire varier le learning rate. Ca a l'air de bien marché. On pourrait tester un nombre de cycle inférieur.
* Ajout du swish, codé sur tensorflow.
* J'ai changé quelques paramètres.

A faire:
* On devrait faire un tableau excel des paramètres à modifier.
* Le tensorboard ne marche pas, voir pourquoi?
* Essayer des conv2D avec un kernel de 5x5?
* Tester le scheduler
* Implémenter un checkpoint daans le callback + un enregistrement automatique en fct du score de précision (pas réussi à le faire encore)
* Tester mobilenetV3.
* Est-ce que plus on augmente le nombre de blocks (bottleneck) mieux les resultats seront? A checker dans les notes de Cazou.
* Sur mobilenetv2, Cazou n'a pas parlé de la résolution et de la width alpha. Il ne me semble pas que cela soit implémenter. A tester.
* Il serait intéressant de tester notre algo avec différentes tailles de batch.
* Tester le transfer learning!!!!!!

In [18]:
!pip install tensorflow-addons

     |████████████████████████████████| 1.1 MB 4.2 MB/s 


In [17]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
# Unzip files
!wget https://www.lamsade.dauphine.fr/~cazenave/project2021.zip
!unzip project2021.zip

--2022-02-11 21:25:55--  https://www.lamsade.dauphine.fr/~cazenave/project2021.zip
Resolving www.lamsade.dauphine.fr (www.lamsade.dauphine.fr)... 193.48.71.250
Connecting to www.lamsade.dauphine.fr (www.lamsade.dauphine.fr)|193.48.71.250|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 138683868 (132M) [application/zip]
Saving to: ‘project2021.zip’

project2021.zip     100%[===================>] 132.26M  11.4MB/s    in 13s     

2022-02-11 21:26:10 (10.1 MB/s) - ‘project2021.zip’ saved [138683868/138683868]

Archive:  project2021.zip
  inflating: Board.h                 
  inflating: Game.h                  
  inflating: Rzone.h                 
  inflating: compile.sh              
  inflating: compileMAC.sh           
  inflating: games.data              
  inflating: golois.cpp              
  inflating: golois.cpython-37m-x86_64-linux-gnu.so  
  inflating: golois.cpython-38-x86_64-linux-gnu.so  
  inflating: golois.py               
  inflating: importGoloi

# Run the python script

## Librairies

In [19]:
# Standard libraries
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
from tensorflow.keras import layers 
from tensorflow.keras import regularizers
import matplotlib.pyplot as plt
import datetime
import os
from keras.callbacks import Callback
from keras.backend import sigmoid
from keras.utils.generic_utils import get_custom_objects
from keras.layers import Activation
from keras import backend
from matplotlib import pyplot
from math import pi
from math import cos
from math import floor
from keras.callbacks import ModelCheckpoint
import tensorflow_addons as tfa


# Custom libraries
import golois

## Python script

***Global functions***

In [5]:
# define swish activation function

def swish(x, beta = 1):
    return (x * sigmoid(beta * x))

# Next we register this custom object with Keras. For this we get our custom objects, tell it to update, 
#then pass in a dictionary with a key of what we want to call it and the activation function for it. 
#Note here we pass the swish function into the Activation class to actually build the activation function.
get_custom_objects().update({'swish': Activation(swish)})

### Squeeze and Excitation block implementation

https://blog.paperspace.com/channel-attention-squeeze-and-excitation-networks/ (explications + codes)

https://francescopochetti.com/a-visual-deep-dive-into-the-building-blocks-of-mobilenetv3/

https://github.com/DrSlink/mobilenet_v3_keras/blob/master/mobilenet_v3.py

In [6]:
def SE_block(in_block, ch, ratio=16):
    """ Squeeze and excitation Block

    Parameters:
    =================================
    in_block: tf.keras.Conv2D, current convolution layer
    ch: int, current number of channels
    ratio: int, squeezing ratio (by default 16 is a good ratio)
    """

    # Squeeze: Global information embedding
    x = layers.GlobalAveragePooling2D()(in_block)

    # Excitation: Adaptive Feature Recalibration
    x = layers.Dense(ch//ratio, activation='relu')(x)
    x = layers.Dense(ch, activation='sigmoid')(x)
    return layers.multiply([in_block, x])



In [7]:
# https://blog.paperspace.com/channel-attention-squeeze-and-excitation-networks/

def squeeze_and_excitation_block(input_X, out_dim, reduction_ratio=16, layer_name='SE-block'):
    """Squeeze-and-Excitation (SE) Block
    SE block to perform feature recalibration - a mechanism that allows
    the network "to perform feature recalibration, through which it can
    learn to use global information to selectively emphasise informative
    features and suppress less useful ones
    """


    with tf.name_scope(layer_name):   # This context manager pushes a name scope, which will make the name of all operations added within it have a prefix.

        # Squeeze: Global Information Embedding
        squeeze = tf.nn.avg_pool(input_X, ksize=[1, *input_X.shape[1:3], 1], strides=[1, 1, 1, 1], padding='VALID', name='squeeze')

        # Excitation: Adaptive Feature Recalibration
        ## Dense (Bottleneck) -> ReLU
        with tf.compat.v1.variable_scope(layer_name+'-variables'):
            excitation = tf.compat.v1.layers.dense(squeeze, units=out_dim/reduction_ratio, name='excitation-bottleneck')
        excitation = tf.nn.relu(excitation, name='excitation-bottleneck-relu')

        ## Dense -> Sigmoid
        with tf.compat.v1.variable_scope(layer_name+'-variables'):
            excitation = tf.compat.v1.layers.dense(excitation, units=out_dim, name='excitation')
        excitation = tf.nn.sigmoid(excitation, name='excitation-sigmoid')

        # Scaling
        scaler = tf.reshape(excitation, shape=[-1, 1, 1, out_dim], name='scaler')

        return input_X * scaler

In [8]:
# Hard swish

def Hswish(inp):
    return inp * tf.nn.relu6(inp + 3.) / 6

#Note here we pass the swish function into the Activation class to actually build the activation function.
get_custom_objects().update({'Hswish': Activation(Hswish)})

In [9]:

# define custom learning rate schedule
class CosineAnnealingLearningRateSchedule(Callback):
	# constructor
	def __init__(self, n_epochs, n_cycles, lrate_max, verbose=0):
		self.epochs = n_epochs
		self.cycles = n_cycles
		self.lr_max = lrate_max
		self.lrates = list()
 
	# calculate learning rate for an epoch
	def cosine_annealing(self, epoch, n_epochs, n_cycles, lrate_max):
		epochs_per_cycle = floor(n_epochs/n_cycles)
		cos_inner = (pi * (epoch % epochs_per_cycle)) / (epochs_per_cycle)
		return lrate_max/2 * (cos(cos_inner) + 1)
 
	# calculate and set learning rate at the start of the epoch
	def on_epoch_begin(self, epoch, logs=None):
		# calculate learning rate
		lr = self.cosine_annealing(epoch, self.epochs, self.cycles, self.lr_max)
		# set learning rate
		backend.set_value(self.model.optimizer.lr, lr)
		# log value
		self.lrates.append(lr)


In [10]:
class LogAnnealingLR(_LRScheduler):
    def __init__(self, optimizer, T_max, eta_min=0, last_epoch=-1):
        self.T_max = T_max
        self.eta_min = eta_min
        super(LogAnnealingLR, self).__init__(optimizer, last_epoch)
    def get_lr(self):
        return [self.eta_min + (base_lr - self.eta_min) *
                (1 + math.cos(math.pi * self.last_epoch / self.T_max)) / 2
                for base_lr in self.base_lrs]

NameError: ignored

***Class GoSaveTheQueen***

In [33]:
class GoSaveTheQueen:
  #Indentations chelous non ?

    def __init__(self, planes, moves, N, epochs, batch, filters, trunk, blocks, model_save_path):
        """
        Initialise un objet de la classe GoSaveTheQueen.
        Un objet de cette classe représente ??


        params
        -------

        """
        self.planes = planes  # input de taille 19x19
        self.moves = moves
        self.N = N # Nombre d'états = nombre d'exemples
        self.epochs = epochs # à modifier - estimer le temps par epoch
        self.batch = batch # à modifier aussi - meilleure taille de batch?
        self.filters = filters # par couche de convolutions
        self.trunk = trunk
        self.blocks = blocks
        self.model_save_path = model_save_path
        print("All the variables have been set")
        self.input_data = np.random.randint(2, size=(self.N, 19, 19, self.planes))
        self.input_data = self.input_data.astype ('float32')
        self.policy = np.random.randint(self.moves, size=(self.N,))
        self.policy = keras.utils.to_categorical (self.policy)
        self.value = np.random.randint(2, size=(self.N,))
        self.value = self.value.astype ('float32')
        print("policy and vale defined")
        self.end = np.random.randint(2, size=(self.N, 19, 19, 2))
        self.end = self.end.astype ('float32')
        self.groups = np.zeros((self.N, 19, 19, 1))
        self.groups = self.groups.astype ('float32')

        print ("getValidation", flush = True) 
        golois.getValidation (self.input_data, self.policy, self.value, self.end)  
        print("End of getValidation")
    
    def bottleneck_block(self, x):
      
        """
        description

        params
        -----
        x : 

        returns
        ----
         : 
        """
        m = layers.Conv2D(self.filters, (1,1),
        kernel_regularizer=regularizers.l2(0.0001), use_bias = False)(x)
        m = layers.BatchNormalization()(m)
        m = layers.Activation("Hswish")(m)
        # m = layers.Activation(tf.nn.swish(x))(m)
        m = layers.DepthwiseConv2D((5,5), padding="same",  # Depthwise step
        kernel_regularizer=regularizers.l2(0.0001),
        use_bias = False)(m)
        m = layers.BatchNormalization()(m)
        m = SE_block(m, ch=64)    # TODO: add a squeeze and excitation block
        m = layers.Activation("Hswish")(m)
        # m = layers.Activation(tf.nn.swish(x))(m)
        m = layers.Conv2D(self.trunk, (1,1),
        kernel_regularizer=regularizers.l2(0.0001),
        use_bias = False)(m)
        m = layers.BatchNormalization()(m)
        return layers.Add()([m, x])

    def getModel_mobilenet (self):
        """
        description

        params
        -----
        x : 

        returns
        ----
         : 
        """                
        input = keras.Input(shape=(19, 19, self.planes), name="board")
        x = layers.Conv2D(self.trunk, 1, padding="same",
        kernel_regularizer=regularizers.l2(0.0001))(input)
        x = layers.BatchNormalization()(x)
        x = layers.Activation("swish")(x)
        # x = layers.ReLU()(x)
        for i in range (self.blocks):
            x = self.bottleneck_block (x)
        policy_head = layers.Conv2D(1, 1, activation="swish", padding="same",
        use_bias = False,
        kernel_regularizer=regularizers.l2(0.0001))(x)
        policy_head = layers.Flatten()(policy_head)
        policy_head = layers.Activation("softmax", name="policy")(policy_head)
        value_head = layers.GlobalAveragePooling2D()(x)
        value_head = layers.Dense(50, activation="swish",
        kernel_regularizer=regularizers.l2(0.0001))(value_head)
        value_head = layers.Dense(1, activation="sigmoid", name="value",
        kernel_regularizer=regularizers.l2(0.0001))(value_head)
        model = keras.Model(inputs=input, outputs=[policy_head, value_head])
        return model
    
    def getModel_resnet(self):
        """
        description

        params
        -----
        x : 

        returns
        ----
         : 
        """
        initializer = tf.keras.initializers.LecunNormal()
        input = keras.Input(shape=(19, 19, self.planes), name='board')
        x = layers.Conv2D(self.filters, 1, activation='swish', padding='same')(input)
        for i in range (6):
            x1 = layers.Conv2D(self.filters, 3, activation='swish', padding='same', kernel_initializer=initializer)(x) # 6 couches de resnet
            x1 = layers.Conv2D(self.filters, 3, padding='same')(x1)
            x = layers.add([x1,x])
            x = layers.Activation("swish")(x)
            # x = layers.ReLU()(x)
            x = layers.BatchNormalization()(x) # add dropout
        policy_head = layers.Conv2D(1, 1, activation='swish', padding='same', use_bias = False, kernel_regularizer=regularizers.l2(0.0001), kernel_initializer=initializer)(x)
        policy_head = layers.Flatten()(policy_head)
        policy_head = layers.Activation('softmax', name='policy')(policy_head)
        value_head = layers.GlobalAveragePooling2D()(x)
        value_head = layers.Dense(50, activation='swish', kernel_regularizer=regularizers.l2(0.0001), kernel_initializer=initializer)(value_head)
        value_head = layers.Dense(1, activation='sigmoid', name='value', kernel_regularizer=regularizers.l2(0.0001))(value_head)
        model = keras.Model(inputs=input, outputs=[policy_head, value_head])
        return model


    def run_epochs(self, model):
        """
        Cette méthode compile et entraine le modèle
    
        params
        -----
        model : 
    
        returns
        ----
         : 
        """
        ### optimizer = tfa.optimizers.AdaBelief(lr=1e-3)
        ### optimizer=keras.optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Nadam", **kwargs)
        ### optimizer=keras.optimizers.SGD(learning_rate=5.0e-8, momentum=0.9)
        opt = tf.keras.optimizers.SGD(learning_rate=5.0e-8, momentum=0.9)
        opt = tfa.optimizers.Lookahead(opt)
        model.compile(optimizer = opt,     # SGD utilisé en tant qu'optimizer. Si entraînement pour 1 million de données, on peut essayer Adam.
                      # reponse : Au contraire, le prof a dit que pour un "very long run" SGD était meilleur ! :)
                      loss={'policy': 'categorical_crossentropy', 'value': 'binary_crossentropy'},
                      loss_weights={'policy' : 1.0, 'value' : 1.0},
                      metrics={'policy': 'categorical_accuracy', 'value': 'mse'})

        filepath="/content/gdrive/MyDrive/Models/weights.best.hdf5"

        #checkpoint = ModelCheckpoint(filepath, monitor='policy_categorical_accuracy', verbose=1,save_best_only=True, mode='max')
        #callbacks_list = [checkpoint]

        # Create a callback that saves the model's weights -> comment on s'en sert ensuite ? c'est si l'entrainement s'arrete en cours ?
        #cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
        #                                         save_weights_only=True,
        #                                        verbose=1)

        # define learning rate callback
        n_cycles = 8
        ca = CosineAnnealingLearningRateSchedule(self.epochs, n_cycles, 0.01)

        best_acc = 0
        for i in range (1, self.epochs + 1):
            print ('epoch ' + str (i))
            # getBatch à expliciter : à chaque k iteration, modifie les données (donne un nouveau batch i en prenant les N données suivantes) ====> Pas de validation set, à ajouter! 
            golois.getBatch (self.input_data, self.policy, self.value, self.end, self.groups, i * self.N)

            history = model.fit(self.input_data,
                        {'policy': self.policy, 'value': self.value}, 
                        epochs=1,
                        batch_size=self.batch)

            if (i % 10 == 0):
                golois.getValidation (self.input_data, self.policy, self.value, self.end) # getValidation: tous les 10-ième itérations, evalue le modèle avec un dataset de validation indépendant du jeu d'entraînement. 
                val = model.evaluate (self.input_data,
                                      [self.policy, self.value], verbose = 0, batch_size=self.batch)
                print ("val =", val)
            if (i % 5 == 0):
                golois.getValidation (self.input_data, self.policy, self.value, self.end) # getValidation: tous les 10-ième itérations, evalue le modèle avec un dataset de validation indépendant du jeu d'entraînement. 
                val = model.evaluate (self.input_data,
                                      [self.policy, self.value], verbose = 0, batch_size=self.batch)
                if val[3] > best_acc:
                    best_acc = val[3]
                    print("saving model  with best accuracy: ", best_acc)
                    model.save(self.model_save_path)

        # plot learning rate
        pyplot.plot(ca.lrates)
        pyplot.show()

        return model, history


### Mobilenet with transformers

## Start main

### 1. Instanciate the class

In [34]:
if __name__ == "__main__":
    print("Start the code")

    planes = 31  # input de taille 19x19
    moves = 361
    N = 10000 # Nombre d'états = nombre d'exemples
    epochs = 200 # à modifier - estimer le temps par epoch
    batch = 256 # à modifier aussi - meilleure taille de batch?
    filters = 64 # par couche de convolutions
    trunk = 128
    blocks= 24
    model_save_path='/content/gdrive/MyDrive/Models/GOsaveTheQueen_v11.h5'

    print("*************Instanciate the class*******************")
    gogo = GoSaveTheQueen(planes, moves, N, epochs, batch, filters, trunk, blocks, model_save_path)
    model = gogo.getModel_mobilenet()
    model.summary ()



Start the code
*************Instanciate the class*******************
All the variables have been set
policy and vale defined
getValidation
End of getValidation
Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 board (InputLayer)             [(None, 19, 19, 31)  0           []                               
                                ]                                                                 
                                                                                                  
 conv2d_214 (Conv2D)            (None, 19, 19, 128)  4096        ['board[0][0]']                  
                                                                                                  
 batch_normalization_311 (Batch  (None, 19, 19, 128)  512        ['conv2d_214[0][0]']             
 Normalization)                

### 2. Training

In [ ]:
print("Compiling the code")
model, history = gogo.run_epochs(model)


Compiling the code
epoch 1
40/40 [==============================] - 40s 294ms/step - loss: 21.1370 - policy_loss: 19.4498 - value_loss: 1.2644 - policy_categorical_accuracy: 0.0034 - value_mse: 0.2309
epoch 2
40/40 [==============================] - 12s 294ms/step - loss: 21.0653 - policy_loss: 19.3942 - value_loss: 1.2484 - policy_categorical_accuracy: 0.0032 - value_mse: 0.2237
epoch 3
40/40 [==============================] - 12s 297ms/step - loss: 21.1797 - policy_loss: 19.5021 - value_loss: 1.2548 - policy_categorical_accuracy: 0.0023 - value_mse: 0.2274
epoch 4
40/40 [==============================] - 12s 294ms/step - loss: 21.1554 - policy_loss: 19.4953 - value_loss: 1.2373 - policy_categorical_accuracy: 0.0023 - value_mse: 0.2244
epoch 5
40/40 [==============================] - 12s 294ms/step - loss: 20.9171 - policy_loss: 19.2668 - value_loss: 1.2275 - policy_categorical_accuracy: 0.0033 - value_mse: 0.2203
saving model  with best accuracy:  9.999999747378752e-05


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


epoch 6
40/40 [==============================] - 12s 294ms/step - loss: 20.9782 - policy_loss: 19.3036 - value_loss: 1.2517 - policy_categorical_accuracy: 0.0040 - value_mse: 0.2231
epoch 7
40/40 [==============================] - 12s 294ms/step - loss: 20.9274 - policy_loss: 19.2780 - value_loss: 1.2267 - policy_categorical_accuracy: 0.0025 - value_mse: 0.2221
epoch 8
40/40 [==============================] - 12s 294ms/step - loss: 20.9434 - policy_loss: 19.2824 - value_loss: 1.2382 - policy_categorical_accuracy: 0.0029 - value_mse: 0.2224
epoch 9
40/40 [==============================] - 12s 294ms/step - loss: 20.8534 - policy_loss: 19.2090 - value_loss: 1.2216 - policy_categorical_accuracy: 0.0034 - value_mse: 0.2204
epoch 10
40/40 [==============================] - 12s 294ms/step - loss: 20.8449 - policy_loss: 19.1870 - value_loss: 1.2351 - policy_categorical_accuracy: 0.0031 - value_mse: 0.2211
val = [7.282537460327148, 6.045233249664307, 0.8145307898521423, 0.001500000013038516, 0.

# Save the model

In [ ]:
model.save('/content/gdrive/MyDrive/Models/GOsaveTheQueen_v2.h5')

In [ ]:


filepath="weights.best.hdf5"

checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1,save_best_only=True, mode='max')
callbacks_list = [checkpoint]

# Load the model

In [ ]:
planes = 31  # input de taille 19x19
moves = 361
N = 40000 # Nombre d'états = nombre d'exemples
epochs = 24 # à modifier - estimer le temps par epoch
batch = 256 # à modifier aussi - meilleure taille de batch?
filters = 64 # par couche de convolutions
trunk = 128
blocks= 15
model_save_path='/content/gdrive/MyDrive/Models/GOsaveTheQueen_v8.h5'
gogo = GoSaveTheQueen(planes, moves, N, epochs, batch, filters, trunk, blocks, model_save_path)
model2 = gogo.getModel_mobilenet()



All the variables have been set


NameError: ignored

In [ ]:
model2.load_weights("/content/gdrive/MyDrive/Models/GOsaveTheQueen_v8.h5")

In [ ]:
model2.summary()

NameError: ignored

In [ ]:
model3, history = gogo.run_epochs(model2)

epoch 1


ResourceExhaustedError: ignored